# INM706 - Deep Learning for Sequence Analysis

Authors: Laerte Adami - Elisa Troschka

In [9]:
from Utilities.lstmHandler import EncoderLSTM, DecoderLSTM
from Utilities.modelHandler import LSTModel
from Utilities.LanguageDataset import LanguageDataset
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss as CEL
from torch.optim import Adam

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
dataset = LanguageDataset(data_path="./Data/eng_ita.tsv")

vocabulary_size = dataset.eng_voc_size
embedding_size = 256

bos_token = '<BoS>'
eos_token = '<EoS>'

loss_func = CEL()

In [3]:
trainloader = DataLoader(dataset, batch_size=1)

In [14]:
batch_size = 1
x, y = next(iter(trainloader))
prev_hidden = torch.zeros(1, batch_size, embedding_size, device = device)

In [15]:
x.size()

torch.Size([1, 6, 26664])

In [16]:
emb = nn.Embedding(vocabulary_size, embedding_size)

In [17]:
out = emb(x)

In [18]:
out.size()

torch.Size([1, 6, 26664, 256])

In [19]:
xx = out.view(x.size()[1], out.size()[0],-1)

In [20]:
xx.size()

torch.Size([6, 1, 6825984])

In [7]:
out, hidden = encoder(x)

RuntimeError: input must have 3 dimensions, got 5

In [6]:
encoder = EncoderLSTM(vocabulary_size = vocabulary_size,
                     embedding_size = embedding_size,
                     num_layers = 1, 
                     bidirectional = False)

decoder = DecoderLSTM(vocabulary_size = vocabulary_size,
                     embedding_size = embedding_size,
                     num_layers = 1, 
                     bidirectional = False)

In [4]:
model = LSTModel(encoder = encoder, 
                 decoder = decoder, 
                 encoder_optimizer = Adam(encoder.parameters()), 
                 decoder_optimizer = Adam(decoder.parameters()),
                 loss_function = loss_func, 
                 eos_token = eos_token, 
                 bos_token = bos_token)

In [8]:
print(decoder)

DecoderLSTM(
  (emb): Embedding(100, 30)
  (lstm): LSTM(30, 30, batch_first=True)
  (relu): ReLU()
)


In [6]:
z = torch.zeros(2,3,4)

In [11]:
zz = z[:,-1,:].unsqueeze(1)

In [12]:
zz.size()

torch.Size([2, 1, 4])

In [1]:
import torch.nn as nn

In [4]:
class EncoderLSTM(nn.Module):
    
    def __init__(self, vocabulary_size, embedding_size, num_layers, bidirectional):
        
        super(EncoderLSTM, self).__init__()
        
        self.emb = nn.Embedding(vocabulary_size, embedding_size)
        self.lstm = nn.lstm(input_size = embedding_size, hidden_size = embedding_size, 
                            num_layers = num_layers, batch_first = True, bidirectional = bidirectional)
        

        
    def forward(self, x, prev_hidden):
        
        x = self.emb(x)
        output, hidden = self.lstm(x, prev_hidden)
        
        return output, hidden
    
class DecoderLSTM(nn.Module):
    
    def __init__(self, vocabulary_size, embedding_size, num_layers, bidirectional):

        super(DecoderLSTM, self).__init__()
        
        self.emb = nn.Embedding(vocabulary_size, embedding_size)
        self.lstm = nn.lstm(input_size = embedding_size, hidden_size = embedding_size, 
                            num_layers = num_layers, batch_first = True, bidirectional = bidirectional)
        self.relu = nn.ReLU(inplace=False)
        
    def forward(self, x, prev_hidden):
        
        x = self.relu(x)
        output, hidden = self.lstm(x, prev_hidden)
        
        return output, hidden
        
        
        
        
        
        
        
        

In [36]:
# an Embedding module containing 10 tensors of size 3
embedding = nn.Embedding(20, 3)
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1,2,4,5],[4,3,2,9]])

print(input.size())
print(input)
test = embedding(input)
test

torch.Size([2, 4])
tensor([[1, 2, 4, 5],
        [4, 3, 2, 9]])


tensor([[[ 0.8975,  0.3501, -0.6632],
         [-0.3166, -0.9211,  0.2392],
         [ 0.4596,  0.3739,  0.1299],
         [-2.1523,  0.4012,  1.4271]],

        [[ 0.4596,  0.3739,  0.1299],
         [ 1.2827,  0.1125,  0.1084],
         [-0.3166, -0.9211,  0.2392],
         [ 0.5954, -0.8891,  0.0884]]], grad_fn=<EmbeddingBackward0>)

In [23]:
test.size()

torch.Size([2, 4, 3])

In [29]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
input = [word_to_ix["hello"]]
lookup_tensor = torch.tensor(input, dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.5390,  0.0380, -0.9304,  0.6133, -0.7360]],
       grad_fn=<EmbeddingBackward0>)


In [27]:
word_to_ix

{'hello': 0, 'world': 1}

In [28]:
hello_embed.size()

torch.Size([1, 5])

In [33]:
len(input)

1